In [1]:
pip install lssvr

Note: you may need to restart the kernel to use updated packages.


  ERROR: Could not find a version that satisfies the requirement lssvr (from versions: none)
ERROR: No matching distribution found for lssvr


In [ ]:
conda install python=3.8

In [2]:
import sys

# Prints current Python version
print("Current version of Python is ", sys.version)

Current version of Python is  3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
from lssvr import LSSVR
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from datetime import datetime
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'lssvr'

In [ ]:
# Reading the csv data into a dataframe
df = pd.read_csv("GOOG.csv", keep_default_na = False)
df
df["close_prev"] = df["Close"].shift(1) # Shifting the closing prices by 1 for easier calculation
df["return"] = np.log(df["Close"])-np.log(df["close_prev"]) # Return: diff of the log values
df = df.iloc[1: , :] # Neglecting 1st value as it becomes NaN

In [ ]:
train_len = round(90*len(df['Date'])/100)

In [ ]:
test_len = len(df['Date']) - train_len

In [ ]:
df['Date']

In [ ]:
train_date = list(df['Date'])[train_len]
print(train_date)
test_date = '2021-10-06'
td = datetime.strptime(test_date,'%Y-%m-%d')

In [ ]:
dates = []
for i in range(len(df)):
    dates.append(datetime.strptime(df.iloc[i].Date,'%Y-%m-%d'))

In [ ]:
df['date_f'] = dates

In [ ]:
td

In [ ]:
train_df = df[df.date_f < td]
test_df = df[df.date_f >= td]

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_data = train_df['Close']
test_data = test_df['Close']

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(train_data)
plt.plot(test_data)
plt.show()

In [ ]:
scaler = MinMaxScaler()
train_data = train_data.values.reshape(-1, 1)
test_data = test_data.values.reshape(-1, 1)
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [ ]:
train_data.shape

In [ ]:
#train_data = train_data.values
#test_data = test_data.values

In [ ]:
timesteps = 100

In [ ]:
train_data_timesteps=np.array([[j for j in train_data[i:i+timesteps]] for i in range(0,len(train_data)-timesteps+1)])[:,:,0]
train_data_timesteps.shape

In [ ]:
test_data_timesteps=np.array([[j for j in test_data[i:i+timesteps]] for i in range(0,len(test_data)-timesteps+1)])[:,:,0]
test_data_timesteps.shape

In [ ]:
x_train, y_train = train_data_timesteps[:,:timesteps-1],train_data_timesteps[:,[timesteps-1]]
x_test, y_test = test_data_timesteps[:,:timesteps-1],test_data_timesteps[:,[timesteps-1]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
y_train = y_train.reshape(-1)
y_test =  y_test.reshape(-1)

In [ ]:
model = LSSVR(kernel='rbf', gamma=0.01,C=0.2)
model.fit(x_train, y_train)
y_hat = model.predict(x_test)
print('MSE', mean_squared_error(y_test, y_hat))
print('R2 Score',model.score(x_test, y_test))

In [ ]:
plt.plot(y_test)
plt.plot(y_hat)
plt.show()

In [ ]:
y_hat